In [ ]:
from astropy import wcs
from astropy.io import ascii
from astropy.io import fits
from astropy.table import Table
import numpy as np
import os

# See the SEXTRACOR CELL in the START.ipynb script for a description of the functions

def SExtractor(im_path, sex_path, sex_cfg, save_path):
    
    folder = os.fsencode(im_path)
    
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    
    for item in os.listdir(folder):
        
        item = os.fsdecode(item)
        
        item = im_path + item
        
        os.system("cp " + item + " " + sex_path)
    
    os.system("cd " + sex_path)
    
    sex_folder = os.fsencode(sex_path)
    
    cfg = sex_path + sex_cfg
    
    for item in os.listdir(sex_folder):
        
        item = os.fsdecode(item)
        
        item = sex_path + item
        
        if item.endswith('.new'):
            
            os.system("sextractor " + item + " -c " + cfg)
            
            os.system("mv " + sex_path + "tmp.cat " + item + ".cat")
            
            os.system("mv " + item + ".cat " + save_path)

## BUILDS PSFs ##

def make_PSF(im_path, cat_path, tmp_path, save_path, sex_cfg, psfex_cfg):
    
    tmp_cat_path = tmp_path + 'tmp.cat'
    
    if not os.path.exists(cat_path):
        os.makedirs(cat_path)
    
    if not os.path.exists(save_path):
        os.makedirs(save_path)
        
    folder = os.fsencode(im_path)
    
    for item in os.listdir(folder):
        
        item = os.fsdecode(item)
        
        item = im_path + item
        
        item_name = os.path.splitext(os.path.basename(item))[0]
        
        os.system('sextractor ' + item + ' -c ' + sex_cfg)
        
        os.system('mv ' + tmp_cat_path + ' ' + cat_path + item_name + '.cat')
        
        cat = cat_path + item_name + '.cat'
        
        os.system ('psfex ' + cat + ' -c ' + psfex_cfg)
        
        psf = cat_path + item_name + '.psf'
        
        resi = tmp_path + 'residual_' + item_name + '.fits'
        
        samp = tmp_path + 'sample_' + item_name + '.fits'
        
        os.system ('mv ' + psf + ' ' + save_path)
        
        os.system ('mv ' + resi + ' ' + save_path)
        
        os.system ('mv ' + samp + ' ' + save_path)

## APPLIES PSFs TO THE IMAGES ##

def apply_PSF(im_path, psf_path, sex_cfg, save_path):
    
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    
    folder = os.fsencode(psf_path)
    
    for item in os.listdir(folder):
        
        item = os.fsdecode(item)
        
        if item[-3:] in 'psf':
            
            item = psf_path + item
            
            item_name = os.path.splitext(os.path.basename(item))[0]
            
            im_name = im_path + item_name + '.fits'
            
            os.system('sextractor -PSF_NAME ' + item + ' ' + im_name + ' -c ' + sex_cfg + ' -CATALOG_NAME ' + item_name + '_psfex.cat -CATALOG_TYPE ASCII_HEAD')
            
            os.system("mv " + item_name + '_psfex.cat ' + save_path + item_name + ".cat")

## CORRECT THE WCS HEADERS ##

def wcs_corr(cat_path, im_path, save_path):
    
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    
    folder = os.fsencode(cat_path)
    
    for item in os.listdir(folder):
        
        item = os.fsdecode(item)
        
        item = cat_path + item
        
        item_name = os.path.splitext(os.path.basename(item))[0]
        
        im_name = im_path + item_name + '.fits'
        
        hdu = fits.open(im_name)
        
        head = hdu[0].header
        
        table = Table.read(item, format = 'ascii.sextractor')
        
        xpix = table['X_IMAGE'] #WCS no catálogo que está errado
        
        ypix = table['Y_IMAGE']
        
        wcs_head = wcs.WCS(head) #Aqui meto o header da fits file da imagem correspondente ao catálogo
        
        pixcoords = np.asarray([xpix, ypix]).T #Transforma lista num array
        
        wcoords = wcs_head.all_pix2world(pixcoords, 1).T #Função que faz a correcção dos valores
        
        table['ra'] = wcoords[0] # * units.deg
        
        table['dec'] = wcoords[1] # * units.deg
        
        ascii.write(table, save_path + item_name + '.dat', format = 'commented_header', overwrite = True) #Novo catálogo corrigido